In [1]:
import collections
import csv
import json
import os
import re
import sqlite3

In [2]:
DATA_DIR = "../../data/tmdb-dataset"
MOVIES_DATA = os.path.join(DATA_DIR, "movies_metadata.csv")
KEYWORDS_DATA = os.path.join(DATA_DIR, "keywords.csv")

LOOKUP_DB = os.path.join(DATA_DIR, "lookups.db")

In [3]:
def table_exists(conn, table_name):
    cur = conn.cursor()
    cur.execute("select name from sqlite_master where type='table' and name = ?", 
                [table_name])
    rows = cur.fetchall()
    cur.close()
    return len(rows) > 0


def create_keywords_table(conn):
    if not table_exists(conn, "keywords"):
        cur = conn.cursor()
        create_table = """
            CREATE TABLE keywords(
                mid INTEGER NOT NULL PRIMARY KEY,
                keywords VARCHAR(255) NOT NULL
            )
        """
        cur.execute(create_table)
        cur.close()


def insert_keywords(conn, movie_id, keywords):
    cur = conn.cursor()
    keywords_str = "|".join(keywords)
    insert_sql = """
        INSERT INTO keywords(mid, keywords) VALUES (?, ?)
    """
    cur.execute(insert_sql, [movie_id, keywords_str])
    cur.close()



conn = sqlite3.connect(LOOKUP_DB)
create_keywords_table(conn)

In [4]:
counter = collections.Counter()
skipped = set()
i = 0
with open(KEYWORDS_DATA, "r") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if i % 1000 == 0:
            print("{:d} movie keywords processed".format(i))
        mid = int(row["id"])
        keywords = row["keywords"]
        # remove apostrophe s
        keywords = re.sub(r"([a-z0-9])'s", r"\1s", keywords)
        # remove s apostrophe
        keywords = re.sub(r"s' ([a-z])", r"s \1", keywords)
        # replace d'whatever as dwhatever (example: coup d'etat, cote d'zur)
        keywords = re.sub(r"d'([a-z])", r"d\1", keywords)
        # replace o'whatever as owhatever (example: o'hare)
        keywords = re.sub(r"o'([a-z])", r"o\1", keywords)
        # replace whatever've as whateverve (example: you've)
        keywords = re.sub(r"([a-z])'ve", r"\1ve", keywords)
        # replace 'n' to n in rock 'n' roll
        keywords = keywords.replace(" 'n' ", " n ")
        # replace goa'uld to goauld
        keywords = keywords.replace("goa'uld", "goauld")
        # replace all'italian with allitalian
        keywords = keywords.replace("all'italian", "allitalian")
        # remove \xa0 in words
        keywords = keywords.replace("\\xa0", "")
        # change quotes to standard json
        keywords = keywords.replace("'", "\"")
        names = []
        keywords_json = json.loads(keywords)
        for idname_pair in keywords_json:
            name = idname_pair["name"]
            names.append(name)
            counter[name] += 1
        if len(names) > 0:
            try:
                insert_keywords(conn, mid, names)
                i += 1
            except sqlite3.IntegrityError as e:
                skipped.add(mid)
                i += 1
                continue

print("{:d} movie keywords processed, COMPLETE".format(i))
# print("skipped following duplicate movie_ids:", skipped)
conn.commit()
conn.close()

0 movie keywords processed
1000 movie keywords processed
2000 movie keywords processed
3000 movie keywords processed
4000 movie keywords processed
4000 movie keywords processed
5000 movie keywords processed
6000 movie keywords processed
7000 movie keywords processed
7000 movie keywords processed
8000 movie keywords processed
9000 movie keywords processed
10000 movie keywords processed
10000 movie keywords processed
11000 movie keywords processed
11000 movie keywords processed
12000 movie keywords processed
13000 movie keywords processed
14000 movie keywords processed
14000 movie keywords processed
14000 movie keywords processed
15000 movie keywords processed
15000 movie keywords processed
15000 movie keywords processed
16000 movie keywords processed
16000 movie keywords processed
16000 movie keywords processed
17000 movie keywords processed
17000 movie keywords processed
18000 movie keywords processed
19000 movie keywords processed
19000 movie keywords processed
19000 movie keywords pr

In [5]:
counter.most_common(100)

[('woman director', 3115),
 ('independent film', 1930),
 ('murder', 1308),
 ('based on novel', 835),
 ('musical', 734),
 ('sex', 685),
 ('violence', 651),
 ('nudity', 636),
 ('biography', 629),
 ('revenge', 626),
 ('suspense', 590),
 ('love', 574),
 ('female nudity', 563),
 ('sport', 544),
 ('police', 454),
 ('teenager', 441),
 ('duringcreditsstinger', 440),
 ('sequel', 439),
 ('friendship', 411),
 ('world war ii', 394),
 ('drug', 360),
 ('prison', 351),
 ('stand-up comedy', 351),
 ('high school', 319),
 ('martial arts', 314),
 ('suicide', 312),
 ('rape', 306),
 ('kidnapping', 306),
 ('silent film', 306),
 ('film noir', 305),
 ('family', 295),
 ('serial killer', 293),
 ('monster', 285),
 ('alien', 283),
 ('dystopia', 273),
 ('paris', 268),
 ('new york', 267),
 ('blood', 266),
 ('gay', 261),
 ('short', 261),
 ('marriage', 258),
 ('christmas', 257),
 ('gore', 246),
 ('death', 245),
 ('zombie', 245),
 ('gangster', 243),
 ('small town', 239),
 ('london england', 238),
 ('romance', 237),
 (